In [1]:
#imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging


load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

In [ ]:
#initialize OpenAI 

openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [ ]:
 system_message = "You are a helpful assistant"

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        stream=True

    )

    response = ""
    for chunk in stream:
        delta = chunk.choices[0].delta
        if 'content' in delta:
            response += delta.content
            yield response




In [ ]:
gr.ChatInterface(fn=chat, type="message").launch()

In [ ]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [ ]:
def chat(message, history):

    relevant_system_message = system_message
    if 'belt' in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()